**Perplexity（困惑度）**

语言模型“平均分岔数”的度量：越小越好。直观地说，PPL ≈ 模型在每个位置上“平均有多少个等可能的选择”。

给定长度为 $T$$ 的标注序列 $x_{1:T}$（通常是 token 序列），自回归 LM 的对数似然
$$log p(x_{1:T})=\sum_{t=1}^{T}\log p(x_t\mid x_{<t})$$

交叉熵 / 负对数似然（单位：nats）
$$\text{NLL} = -\frac{1}{T}\sum_{t=1}^{T}\log p(x_t\mid x_{<t})$$

Perplexity（自然对数制）
$$\mathrm{PPL}=\exp(\text{NLL})$$

如果用的是 以 2 为底的交叉熵（bits-per-token, BPT）
$$\mathrm{PPL}=2^{\text{BPT}}$$

两者关系
$$\text{BPT}=\text{NLL}/\ln 2$$

例子：平均 NLL = 1.2（nats）⇒ PPL = e^{1.2}\approx 3.32。
若 BPT = 1（每个 token 1 bit）⇒ PPL = 2^1=2。

**训练和评测时怎么算**
- **教师强制（teacher forcing）** 前向，拿到每个位置的预测分布；
- 取目标 token 的 log 概率，累加得到总 NLL；
- 用有效 token 数（排除 padding、被忽略标签）做平均：$\text{NLL}_\text{avg}=\text{NLL}_\text{sum}/T_\text{valid}$；
- PPL = $exp(NLL_avg)$；

**注意事项**
- 如果用 HuggingFace 模型直接返回的 loss（默认是 mean），需乘上本 batch 的有效 token 数再累计，最后除以全局 token 数再 exp；
- token 粒度：PPL 与 分词器强相关（BPE / word / char）；跨模型 / 语料比较时要一致的分词方案；
- 序列 / 批次加权：要按 token 总数加权，不能对 batch 级均值再做简单平均；
- 忽略项：padding、被 mask 的标签（-100）不应计入分母；
- 大多数深度学习框架的 CrossEntropy 使用自然对数，所以 ppl = exp(loss)；若你算的是 bits-per-token，就用 2**loss_bits；
- 带标签平滑 / 类目加权时 PPL 不再等价于真实对数似然，比较需谨慎；
- BERT 等双向 MLM 没有标准 PPL，可用 pseudo-perplexity（逐位 mask 评估）或改造成自回归评测；
- PPL ≈ 1 几乎每步都非常确定（理想极限）；PPL ≈ 词表大小模型基本瞎猜；
- 在同一数据与分词设定下，更低的 PPL 通常意味着更好的整体语言建模，但对下游生成质量仍需结合任务指标（如 Rouge / BLEU / human eval）；

**PPL 以上的能力面**
现代大模型评测更像一整套“可用性 + 可靠性 + 成本”的体检。

- 能力面
	- 通识与推理：MMLU、HellaSwag、Winogrande、ARC；指标 = Accuracy / Exact-Match；
	- 数学：GSM8K、MATH；指标 = Accuracy、一步步推导是否自洽（用判据 / judge）；
	- 代码：HumanEval、MBPP、竞赛题；指标 = pass@k（建议 k= 1 / 5 / 10）；
	- 多轮与规划：多轮任务完成率、步骤一致性、死循环率；
	- 多语种 / 多模态：跨语言一致性、翻译质量（COMET / BLEU）、图表理解正确率；

- 真实性与可溯源
	- 事实一致性 / 幻觉率：支持证据的比例、无依据断言占比（#unsupported claims / #claims）。
	- 带检索 / 长文问答（RAG）：答案与上下文的相关性 / 覆盖率 / 精确度（可用 RAGAS：answer-relevancy、context-precision / recall、groundedness）；
	- 引用合规：是否返回可点击来源，引用与答案是否匹配；
	- 数据污染检查：评测集与训练数据重叠率（n-gram / 哈希近邻）；

- 指令遵循与工具使用
	- 格式遵循：JSON 有效率、Schema 合格率、函数调用参数正确率；
	- 工具 / RAG / 函数路由：是否在需要时调用工具、调用是否最小化次数、失败重试率；
	- 拒答与覆盖：该拒绝时拒绝率、应答时命中率（coverage）；

- 长上下文与记忆
	- 长文利用：needle-in-a-haystack 命中率、跨段引用正确率；
	- 位置鲁棒：lost-in-the-middle 测试（信息放前 / 中 / 后的一致性）；
	- KV Cache 质量：极长生成的退化点、重复 / 游走率；

- 安全、偏见与隐私
	- 安全对抗：越狱/Jailbreak 套件通过率、提示注入抵抗力。
	- 有害内容：毒性/仇恨/违法输出率（可对接外部判别器）。
	- 偏见/公平：不同群体/语言/方言的一致性（StereoSet 等）。
	- 隐私：PII 泄露率、训练样本记忆测验（membership inference）。

- 稳定性与鲁棒性
	- 提示敏感度：同义改写 / 拼写噪声 / 格式扰动下性能跌幅；
	- 校准：置信度–正确率一致性（ECE、Brier Score）、可选择性回答（coverage-risk 曲线）；
	- 再现性：多随机种子方差、温度变化敏感度；

- 体验与人评
	- 人类偏好：A / B 盲评胜率、ELO（如 Arena 风格）、帮助度 / 无害度 / 真实性三分法；
	- 对话体验：平均回合数、澄清次数、被动重复率、内容多样性（去重 n-gram 比例）；

- 效率与成本
	- 延迟：TTFT / TTFTokens、p50 / p95；
	- 吞吐：tokens/s、并发下退化曲线；
	- 资源：显存占用、KV 内存随长度增长斜率、每 1k tokens 成本；
	- 稳定性：长生成超时率、中断 / 重试率；

In [23]:
import torch
ce = torch.nn.CrossEntropyLoss(ignore_index=-100, reduction='sum')
vocab_size = 10
model = torch.nn.Linear(100, vocab_size)

class DummyDataset(torch.utils.data.Dataset):
    def __init__(self, seq_len=10):
        self.seq_len = seq_len

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return {
            'input_ids': torch.randint(0, vocab_size, (self.seq_len,), dtype=torch.float32),
            'attn_mask': torch.ones(self.seq_len),
            'labels': torch.randint(0, vocab_size, (self.seq_len,))
        }

dataset = DummyDataset(seq_len=100)
loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

total_nll, total_tok = 0.0, 0
model.eval()

with torch.no_grad():
    for batch in loader:
        input_ids = batch['input_ids']
        batch_size, seq_len = input_ids.shape
        hidden = input_ids.view(batch_size * seq_len, -1)
        hidden = hidden.mean(dim=1, keepdim=True).expand(-1, 100)
        logits_flat = model(hidden)
        logits = logits_flat.view(batch_size, seq_len, vocab_size)
        nll = ce(logits.view(-1, logits.size(-1)), batch['labels'].view(-1))
        total_nll += nll.item()
        total_tok += (batch['labels'] != -100).sum().item()

ppl = torch.exp(torch.tensor(total_nll / total_tok)).item()
print("PPL =", ppl)

PPL = 18.65326690673828
